<a href="https://colab.research.google.com/github/itsbhoomika/Clarification-Seeking-LLMs-CS546/blob/main/Copy_of_dataset_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# # Choose model
# model_name = "Qwen/Qwen2.5-7B-Instruct"

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)

# # Move to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",          # let Accelerate place layers
    torch_dtype=torch.bfloat16, # good precision for A100/T4
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

ds = load_dataset("sewon/ambig_qa", "light")

README.md: 0.00B [00:00, ?B/s]

light/train-00000-of-00001.parquet:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

light/validation-00000-of-00001.parquet:   0%|          | 0.00/367k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10036 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2002 [00:00<?, ? examples/s]

In [ ]:
data = ds['train']
print(len(data))

10036


In [ ]:
data[1]

{'id': '4790842463458965203',
 'question': 'Who played george washington in the john adams series?',
 'annotations': {'type': ['singleAnswer'],
  'answer': [['David Morse']],
  'qaPairs': [{'question': [], 'answer': []}]}}

In [ ]:
data[0]

{'id': '-4469503464110108318',
 'question': 'When did the simpsons first air on television?',
 'annotations': {'type': ['multipleQAs'],
  'answer': [[]],
  'qaPairs': [{'question': ['When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?',
     'When did the Simpsons first air as a half-hour prime time show?'],
    'answer': [['April 19, 1987'], ['December 17, 1989']]}]}}

For golf's dataset importing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import json

# file_path = 'ambignq_train_data.json' # READ!!! THIS HAS TO BE YOUR PATH (GET THE DATA BY YOURSELF AMBIGQA)
# with open(file_path, 'r') as f:
#     data = json.load(f)

# print(len(data))

end golf's importing

In [ ]:
import pandas as pd
import json

# --- 1. Load your data (if not already loaded) ---
# data = [json.loads(line) for line in open("ambignq_train_data.json")]

rows = []
for entry in data:
    ann = entry["annotations"]

    # type
    ann_type = ann["type"][0] if isinstance(ann["type"], list) else ann["type"]

    # qaPairs (may be nested)
    qa_pairs = ann.get("qaPairs", [])

    # flatten one extra layer if needed
    if len(qa_pairs) == 1 and isinstance(qa_pairs[0], list):
        qa_pairs = qa_pairs[0]

    row = {
        "question": entry["question"],
        "type": ann_type,
        "qaPairs": qa_pairs
    }
    rows.append(row)

df = pd.DataFrame(rows)


In [ ]:
def extract_disambig_questions(qa_pairs):
    disambig = []
    if not isinstance(qa_pairs, list):
        return disambig
    for pair in qa_pairs:
        if not isinstance(pair, dict):
            continue
        qs = pair.get("question", [])
        if isinstance(qs, list):
            disambig.extend(qs)
        elif isinstance(qs, str):
            disambig.append(qs)
    return disambig

df["disambig_questions"] = df["qaPairs"].apply(extract_disambig_questions)


In [ ]:
df["is_ambiguous"] = df["type"].apply(lambda t: t == "multipleQAs")

In [ ]:
df=df[:20] #whole df
df

,question,type,qaPairs,disambig_questions,is_ambiguous
0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,[When did the Simpsons first air on television...,True
1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[],False
2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,"[What is the legal age of marriage, without pa...",True
3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,[Who starred in barefoot in the park on broadw...,True
4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,"[Based on the initial thoughts of the project,...",True
5,When did the frozen ride open at epcot?,singleAnswer,"[{'question': [], 'answer': []}]",[],False
6,Name the landforms that form the boundaries of...,singleAnswer,"[{'question': [], 'answer': []}]",[],False
7,When was the last time uga won a national cham...,multipleQAs,[{'question': ['When was the last time UGA won...,[When was the last time UGA won a national foo...,True
8,Who sing play that funky music white boy?,multipleQAs,[{'question': ['Who sing play that funky music...,[Who sing play that funky music white boy in 1...,True
9,When was the first airplane used in war?,singleAnswer,"[{'question': [], 'answer': []}]",[],False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.to_csv('/content/drive/MyDrive/cs546_ambigqa.csv', index=False)

In [ ]:
# import pandas as pd
# pd.set_option('display.max_colwidth', None)

# rows = []
# for entry in data:
#     ann = entry['annotations'][0]
#     row = {
#         'question': entry['question'],
#         'type': ann['type'],
#         'qaPairs': ann.get('qaPairs', None)  # only present for multipleQAs
#     }
#     rows.append(row)

# df = pd.DataFrame(rows)
# df['disambig_questions'] = df['qaPairs'].apply(
#     lambda pairs: [q['question'] for q in pairs] if isinstance(pairs, list) else []
# )
# df['is_ambiguous'] = df['type'] == 'multipleQAs'

In [ ]:
# df = df[:20] # I dont have the GPU lmao
# df

In [ ]:
# import re
# import ast

# def extract_facets(text):
#     """
#     Extracts the values enclosed within [FACET]...[/FACET] tags.
#     Example input:  '[FACET]["tennis", "golf"][/FACET]'
#     Output: ['tennis', 'golf']
#     """
#     if not isinstance(text, str):
#         return []

#     match = re.search(r'\[FACET\](.*?)\[/FACET\]', text, re.DOTALL)
#     if not match:
#         return []

#     facet_str = match.group(1).strip()
#     try:
#         # Safely evaluate a Python-style list, e.g. ["tennis", "golf"]
#         values = ast.literal_eval(facet_str)
#         if isinstance(values, list):
#             return [str(v).strip() for v in values]
#         else:
#             return [str(values).strip()]
#     except Exception:
#         # Fallback: split manually if it's not a valid list
#         return [v.strip(' "\'') for v in facet_str.split(',') if v.strip()]

# extract_facets("""[FACET]["Corie Bratter", "Victor Velasco", "Mrs. Banks", "Paul Bratter", "Telephone Man", "Delivery Man"][/FACET]""")

In [ ]:
# def extract_label(text: str):
#     """
#     Extracts the first [TYPE]...[/TYPE] value from a string.
#     Returns a single string like "LEXICAL".
#     If none found, returns "N/A".
#     """
#     if not isinstance(text, str):
#         return "N/A"

#     match = re.search(r'\[TYPE\](.*?)\[/TYPE\]', text, re.DOTALL)
#     if match:
#         return match.group(1).strip().upper()
#     return "N/A"

In [ ]:
# facets_results = []

# for index, row in df.iterrows():
#   if not row['is_ambiguous']:
#     facets_results.append('N/A')
#     continue
#   ambig_q_str = row['question']
#   disambig_qs_str = '\n'.join(row['disambig_questions'])

#   # EDIT
#   facet_prompt = f'''
# Given this ambiguous question:
# {ambig_q_str}

# Disambiguated questions:
# {disambig_qs_str}

# Identify the facet-specific information — the minimal added words or concepts that clarify the ambiguity.

# Enclose all extracted facet-specific terms in [FACET][...][/FACET].

# Example:
# Ambiguous: "Who won the US Open?"
# Disambiguated:
# 1. "Who won the US Open in tennis?"
# 2. "Who won the US Open in golf?"
# Output: [FACET]["tennis", "golf"][/FACET]
# Do NOT repeat the question, examples, or any other text.
#   '''
#   inputs = tokenizer(facet_prompt, return_tensors="pt").to(device)
#   # Generate response
#   with torch.no_grad():
#       outputs = model.generate(**inputs, max_new_tokens=100)
#   prompt_len = inputs["input_ids"].shape[1]
#   new_tokens = outputs[0, prompt_len:]
#   text = tokenizer.decode(new_tokens, skip_special_tokens=True)
#   facets_results.append(extract_facets(text))

# df['facets'] = facets_results

In [ ]:
from tqdm import tqdm  # progress bar
import re

FACET PREDICTION

In [ ]:
def extract_facets(text):
    """
    Extract facets from text like:
    [FACET]["Sport Type", "Event Category", "Year"][/FACET]

    Returns: ["Sport Type", "Event Category", "Year"] (as actual list)
    """
    import json

    m = re.search(r'\[FACET\](.*?)\[/FACET\]', text, flags=re.DOTALL | re.IGNORECASE)
    if not m:
        return []  # Return empty LIST, not string

    facet_str = m.group(1).strip()

    try:
        # Try to parse as JSON list
        facets = json.loads(facet_str)
        if isinstance(facets, list):
            return [f.strip() for f in facets]
    except:
        pass

    # Fallback: split by comma
    facets = [f.strip().strip('"').strip("'") for f in facet_str.split(',')]
    return [f for f in facets if f]  # Return LIST, not string

In [ ]:
# facets_results = []

# for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating facets"):
#     if not row['is_ambiguous']:
#         facets_results.append('N/A')
#         continue

#     ambig_q_str = row['question']

#     facet_list = []

#     for disambig_q_str in row['disambig_questions']:

#       facet_prompt = f"""
# You are an expert linguist and QA researcher.
# Your task is to identify the key *facets of ambiguity* that vary across a disambiguated version of a question.
# The facet represents one dimension of variation (e.g., time, location, meaning of a term, type of entity, etc.).

# Follow these instructions carefully:
# 1. Look at the ambiguous question and its disambiguated version.
# 2. Identify what aspects differ between them.
# 3. Output ONLY the list of facet labels enclosed in [FACET]..[/FACET].
# 4. The facet should be short, general, and descriptive (e.g., "Sport Type", "Event Category", "Year", "Location", "Meaning of Term").

# ### FEW-SHOT EXAMPLES ###
# Example 1:
# Ambiguous Question:
# Who won the US Open?

# Disambiguated Question:
# 1. Who won the US Open Tennis Women’s Singles in 2023?
# 2. Who won the US Open Golf Championship in 2019?

# Output:
# [FACET]Sport Type[/FACET]

# Example 2:
# Ambiguous Question:
# When did Fast and Furious 6 come out?

# Disambiguated Question:
# When did Fast and Furious 6 premiere in the UK?

# Output:
# [FACET]Geographical Region[/FACET]

# Example 3:
# Ambiguous Question:
# Where does India rank in the world economy?

# Disambiguated Questions:
# Where does India rank in the world economy according to IMF?

# Output:
# [FACET]Information Source[/FACET]

# ### TASK ###
# Now do the same for the following case.

# Ambiguous Question:
# {ambig_q_str}

# Disambiguated Questions:
# {disambig_q_str}

# Output ONLY the facet list enclosed between [FACET]..[/FACET].
#   """

#       inputs = tokenizer(facet_prompt, return_tensors="pt").to(device)
#       with torch.no_grad():
#           # greedy decoding
#           outputs = model.generate(**inputs, max_new_tokens=200, do_sample=False)
#       prompt_len = inputs["input_ids"].shape[1]
#       new_tokens = outputs[0, prompt_len:]
#       text = tokenizer.decode(new_tokens, skip_special_tokens=True)
#       facet = extract_facets(text)
#       facet_list.append(facet)

#     facets_results.append(facet_list)

# df['facets'] = facets_results


Generating facets: 100%|██████████| 50/50 [07:14<00:00,  8.69s/it]


NEW FACET TEST CODE - SEND ALL QUETIONS AT ONCE AND PICK OUT MISSING FACETS

In [ ]:
facets_results = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating facets"):
    if not row['is_ambiguous']:
        facets_results.append([])  # Empty list, not 'N/A'
        continue

    ambig_q_str = row['question']

    # Join ALL disambiguated questions together
    disambig_qs_str = '\n'.join([f"{i+1}. {q}" for i, q in enumerate(row['disambig_questions'])])

    facet_prompt = f"""
You are an expert linguist and QA researcher.
Your task is to identify the key *facets of ambiguity* that vary across different disambiguated versions of a question.
Each facet represents one dimension of variation (e.g., time, location, meaning of a term, type of entity, etc.).

Follow these instructions carefully:
1. Look at the ambiguous question and ALL its disambiguated versions.
2. Identify what aspects differ between them.
3. Output ONLY the list of facet labels enclosed in [FACET][...][/FACET].
4. Each facet should be short, general, and descriptive (e.g., "Sport Type", "Event Category", "Year", "Location", "Meaning of Term").

### FEW-SHOT EXAMPLES ###
Example 1:
Ambiguous Question:
Who won the US Open?

Disambiguated Questions:
1. Who won the US Open Tennis Women's Singles in 2023?
2. Who won the US Open Golf Championship in 2019?

Output:
[FACET]["Sport Type", "Event Category", "Year"][/FACET]

Example 2:
Ambiguous Question:
When did Fast and Furious 6 come out?

Disambiguated Questions:
1. When did Fast and Furious 6 premiere in the UK?
2. When did Fast and Furious 6 release in the US?

Output:
[FACET]["Meaning of 'come out'", "Geographical Region"][/FACET]

Example 3:
Ambiguous Question:
Where does India rank in the world economy?

Disambiguated Questions:
1. According to IMF, where does India rank?
2. According to World Bank, where does India rank?

Output:
[FACET]["Information Source"][/FACET]

### TASK ###
Now do the same for the following case.

Ambiguous Question:
{ambig_q_str}

Disambiguated Questions:
{disambig_qs_str}

Output ONLY the facet list enclosed between [FACET][...][/FACET].
"""

    inputs = tokenizer(facet_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=200, do_sample=False)

    prompt_len = inputs["input_ids"].shape[1]
    new_tokens = outputs[0, prompt_len:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # extract_facets should return a LIST of facets, e.g., ["Sport Type", "Year"]
    facets = extract_facets(text)  # This should return a list
    facets_results.append(facets)

df['facets'] = facets_results

Generating facets: 100%|██████████| 20/20 [07:59<00:00, 23.97s/it]


In [ ]:
pd.set_option('display.max_colwidth', None)
df

,question,type,qaPairs,disambig_questions,is_ambiguous,facets
0,When did the simpsons first air on television?,multipleQAs,"[{'question': ['When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?', 'When did the Simpsons first air as a half-hour prime time show?'], 'answer': [['April 19, 1987'], ['December 17, 1989']]}]","[When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?, When did the Simpsons first air as a half-hour prime time show?]",True,"[Type of Show, Platform]"
1,Who played george washington in the john adams series?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[]
2,What is the legal age of marriage in usa?,multipleQAs,"[{'question': ['What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?', 'What is the legal age of marriage, without parental consent or other authorization, in Nebraska?', 'What is the legal age of marriage, without parental consent or other authorization, in Mississippi?', 'What is youngest legal age of marriage possible in some US states when circumstances permit?'], 'answer': [['18 years of age', '18'], ['19'], ['21'], ['0']]}]","[What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?, What is the legal age of marriage, without parental consent or other authorization, in Nebraska?, What is the legal age of marriage, without parental consent or other authorization, in Mississippi?, What is youngest legal age of marriage possible in some US states when circumstances permit?]",True,"[State, Circumstances for Marriage]"
3,Who starred in barefoot in the park on broadway?,multipleQAs,"[{'question': ['Who starred in barefoot in the park on broadway as Corie Bratter?', 'Who starred in barefoot in the park on broadway as Victor Velasco?', 'Who starred in barefoot in the park on broadway as Mrs. Banks?', 'Who starred in barefoot in the park on broadway as Paul Bratter?', 'Who starred in barefoot in the park on broadway as Telephone Man?', 'Who starred in barefoot in the park on broadway as Delivery Man?'], 'answer': [['Elizabeth Ashley'], ['Kurt Kasznar'], ['Mildred Natwick'], ['Robert Redford'], ['Herbert Edelman'], ['Joseph Keating']]}]","[Who starred in barefoot in the park on broadway as Corie Bratter?, Who starred in barefoot in the park on broadway as Victor Velasco?, Who starred in barefoot in the park on broadway as Mrs. Banks?, Who starred in barefoot in the park on broadway as Paul Bratter?, Who starred in barefoot in the park on broadway as Telephone Man?, Who starred in barefoot in the park on broadway as Delivery Man?]",True,[Character Name]
4,When did the manhattan project began and end?,multipleQAs,"[{'question': ['Based on the initial thoughts of the project, when did the Manhattan project begin and end?', 'Based on when the project was being worked on, when did the Manhattan project begin and end?'], 'answer': [['Began 1939, end 1946'], ['Began 1942, end 1946']]}]","[Based on the initial thoughts of the project, when did the Manhattan project begin and end?, Based on when the project was being worked on, when did the Manhattan project begin and end?]",True,[Time Frame Interpretation]
5,When did the frozen ride open at epcot?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[]
6,Name the landforms that form the boundaries of the peninsular plateau?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[]
7,When was the last time uga won a national championship?,multipleQAs,"[{'question': ['When was the last time UGA won a national football championship?', 'When was the last time UGA won a national gymnastics championship?', 'When was the last time UGA won a national baseball championship?', 'When was the last time UGA won a national golf championship?', 'When was the last time UGA won a national women's swimming and diving championship?', 'When was the last time UGA 

TAXONOMY PREDICTION

In [ ]:
# taxonomy_labels = []

# for index, row in df.iterrows():
#   if not row['is_ambiguous'] or len(row['facets']) == 0:
#     taxonomy_labels.append('N/A')
#     continue
#   ambig_q_str = row['question']
#   facets = '\n'.join(row['facets'])

#   # EDIT
#   label_prompt = f'''
# Given this ambiguous question:
# {ambig_q_str}

# And its facet-specific information:
# {facets}

# Your task:
# Identify what kind of ambiguity this question represents based on the relationship among its facets.

# Classify the ambiguity using one or more of the following taxonomy categories:
# - LEXICAL: word or phrase with multiple meanings (e.g., "US Open" = tennis vs golf)
# - REFERENTIAL: unclear reference (e.g., "he", "it", "the company")
# - TEMPORAL: uncertainty about time or period
# - SPATIAL: uncertainty about location or place
# - SCOPE: ambiguity in range, condition, or inclusion criteria
# - ENTITY: uncertainty about which entity or group is referred to
# - OTHER: any other type not listed above

# Output the single best label enclosed in [LABEL]...[/LABEL].

# Example:
# Ambiguous: "Who won the US Open?"
# Facets: ["tennis", "golf"]
# Output: [LABEL]LEXICAL[/LABEL]
# Do NOT repeat the question, examples, or any other text.
#   '''
#   inputs = tokenizer(label_prompt, return_tensors="pt").to(device)
#   # Generate response
#   with torch.no_grad():
#       outputs = model.generate(**inputs, max_new_tokens=100)
#   prompt_len = inputs["input_ids"].shape[1]
#   new_tokens = outputs[0, prompt_len:]
#   text = tokenizer.decode(new_tokens, skip_special_tokens=True)
#   taxonomy_labels.append(extract_label(text))

# df['taxonomy'] = taxonomy_labels

In [ ]:
# df=df[:2]

new code for test

In [ ]:
# import re
# import pandas as pd
# from tqdm import tqdm  # progress bar

# # ================================
# # 🔹 Helper Functions
# # ================================

# def extract_reasoning(text):
#     """
#     Extract reasoning section starting with 'Reasoning:' and ending before the [TYPE] tag.
#     """
#     m = re.search(r"Reasoning\s*:\s*(.*?)(?=\[TYPE|\Z)", text, flags=re.DOTALL | re.IGNORECASE)
#     return m.group(1).strip() if m else "N/A"


# def extract_label(text):
#     """
#     Extracts the taxonomy label inside [TYPE]...[/TYPE].
#     Handles cases with or without quotes.
#     """
#     m = re.search(r"\[TYPE\].*?\[/TYPE\]", text, flags=re.DOTALL | re.IGNORECASE)
#     if not m:
#         return "N/A"

#     block = m.group(0)
#     m2 = re.search(r'\[TYPE\]\s*["\']?([^"\']+?)["\']?\s*\[/TYPE\]', block, flags=re.IGNORECASE)
#     if m2:
#         return m2.group(1).strip()

#     # fallback
#     return re.sub(r"\[/?TYPE\]", "", block, flags=re.IGNORECASE).strip()


# def clean_taxonomy_value(t):
#     """
#     Clean up taxonomy strings to remove brackets, quotes, or stray characters.
#     """
#     if isinstance(t, str):
#         return t.replace('[', '').replace(']', '').replace('"', '').replace("'", '').strip()
#     return t


# # ================================
# # 🔹 Taxonomy Classification Loop
# # ================================

# facet_taxonomies_all = []

# for idx, row in tqdm(df.iterrows(), total=len(df)):
#     if not row.get("is_ambiguous") or not isinstance(row.get("facets"), list) or len(row["facets"]) == 0:
#         facet_taxonomies_all.append([])
#         continue

#     ambig_q = row["question"]
#     disambig_qs = "\n".join(row.get("disambig_questions", []))
#     row_facet_results = []

#     for idx, facet in enumerate(row["facets"]):

#         if facet == 'N/A':
#           row_facet_results.append([])
#           continue
#         disambig_q = row['disambig_questions'][idx]
#         label_prompt = f"""
# You are a linguistics expert classifying ambiguity questions and facets according to the given taxonomy definitions.

# Your task:
# 1. Compare the ambiguous question and its disambiguated versions.
# 2. Identify how the facet explains the differences between them.
# 3. Choose ONE taxonomy label from the list below that best describes this ambiguity.
# 4. I want two outputs from you
# 4.1 Your reason - Begin your reasoning with 'Reasoning:' (1–2 sentences).
# 4.2 final taxonomy label - Output the final label once in this exact format:
# [TYPE]["<Taxonomy Name>"][/TYPE]

# Do not output anything else

# Taxonomy Options:
# 1. Entity Reference – Multiple entities share the same name (e.g., "US Open" → tennis vs golf).
# 2. Part of Entity Reference – One entity can have different parts/variations ("Olympics" → Men’s 100 m vs Women’s Marathon).
# 3. Relationships Between Entities – Same entities, different relationships ("Who plays Lara Croft" → child vs adult version).
# 4. Underspecified Common Nouns – Common noun class underspecified ("new album" → live vs remix).
# 5. Degree of an Action – Ambiguity in how much/extent ("When did the gold standard end" → partly vs completely).
# 6. Means of an Action – Ambiguity in how an action is done ("How was the Garbage Patch found" → satellite vs in-person).
# 7. Output Type – Ambiguity in what form the answer takes ("When did the runner-up stop being VP" → year vs event).
# 8. Temporal Dependency – Ambiguity in timeframe ("How long is the term for Texas Governor" → 1876 vs today).
# 9. Geographical Dependency – Ambiguity in region/location ("When was Tomb Raider released" → Berlin vs UK).
# 10. Information Source Dependency – Ambiguity by source ("Where does India rank in the economy" → IMF vs World Bank).

# Now classify this case.


# Ambiguous Question:
# {ambig_q}

# Disambiguated Question:
# {disambig_q}

# Facet:
# {facet}

# """

#         # ---- Generate model response ----
#         inputs = tokenizer(label_prompt, return_tensors="pt").to(device)
#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=100,
#                 temperature=0.0,
#                 do_sample=False,
#             )

#         prompt_len = inputs["input_ids"].shape[1]
#         new_tokens = outputs[0, prompt_len:]
#         text = tokenizer.decode(new_tokens, skip_special_tokens=True)

#         print(f"\n--- FACET: {facet} ---")
#         print(text)
#         print("------------------------\n")

#         reasoning = extract_reasoning(text)
#         taxonomy_label = extract_label(text)

#         row_facet_results.append({
#             "facet": facet,
#             "taxonomy": taxonomy_label,
#             "reasoning": reasoning
#         })

#     facet_taxonomies_all.append(row_facet_results)


# # ================================
# # 🔹 Store in DataFrame
# # ================================

# facet_taxonomies_clean = []
# taxonomy_reasoning_clean = []

# for lst in facet_taxonomies_all:
#     facet_pairs = []
#     reasoning_pairs = []
#     for d in lst:
#         tax = clean_taxonomy_value(d["taxonomy"])
#         facet_pairs.append({
#             "facet": d["facet"],
#             "taxonomy": tax
#         })
#         reasoning_pairs.append({
#             "facet": d["facet"],
#             "taxonomy": tax,
#             "reasoning": d["reasoning"]
#         })
#     facet_taxonomies_clean.append(facet_pairs)
#     taxonomy_reasoning_clean.append(reasoning_pairs)

# df["facet_taxonomies"] = facet_taxonomies_clean          # facet + taxonomy
# df["taxonomy_reasoning"] = taxonomy_reasoning_clean      # facet + taxonomy + reasoning
# df["taxonomy"] = df["facet_taxonomies"].apply(lambda lst: [d["taxonomy"] for d in lst])


with a diff longer prompt (results looks good too)

In [ ]:
import re
import pandas as pd

# ================================
# 🔹 Helper Functions
# ================================

def extract_reasoning(text):
    """
    Extract reasoning section starting with 'Reasoning:' and ending before the [TYPE] tag.
    """
    m = re.search(r"Reasoning\s*:\s*(.*?)(?=\[TYPE|\Z)", text, flags=re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else "N/A"


def extract_label(text):
    """
    Extracts the taxonomy label inside [TYPE]...[/TYPE].
    Handles cases with or without quotes.
    """
    m = re.search(r"\[TYPE\].*?\[/TYPE\]", text, flags=re.DOTALL | re.IGNORECASE)
    if not m:
        return "N/A"

    block = m.group(0)
    m2 = re.search(r'\[TYPE\]\s*["\']?([^"\']+?)["\']?\s*\[/TYPE\]', block, flags=re.IGNORECASE)
    if m2:
        return m2.group(1).strip()

    # fallback
    return re.sub(r"\[/?TYPE\]", "", block, flags=re.IGNORECASE).strip()


def clean_taxonomy_value(t):
    """
    Clean up taxonomy strings to remove brackets, quotes, or stray characters.
    """
    if isinstance(t, str):
        return t.replace('[', '').replace(']', '').replace('"', '').replace("'", '').strip()
    return t


# ================================
# 🔹 Taxonomy Classification Loop
# ================================

facet_taxonomies_all = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    if not row.get("is_ambiguous") or not isinstance(row.get("facets"), list) or len(row["facets"]) == 0:
        facet_taxonomies_all.append([])
        continue

    ambig_q = row["question"]
    disambig_qs = "\n".join(row.get("disambig_questions", []))
    row_facet_results = []

    for idx, facet in enumerate(row["facets"]):

        if facet == 'N/A':
            row_facet_results.append({})
            continue
        disambig_q = row['disambig_questions'][idx]

        label_prompt = f"""
You are a linguistics expert classifying ambiguity questions and facets according to the given taxonomy definitions.

Your task:
1. Compare the ambiguous question and its disambiguated versions.
2. Identify how the facet explains the differences between them.
3. Choose ONE taxonomy label from the list below that best describes this ambiguity.
4. I want two outputs from you
4.1 Your reason - Begin your reasoning with 'Reasoning:' (1–2 sentences).
4.2 final taxonomy label - Output the final label once in this exact format:
[TYPE]<Taxonomy Name>[/TYPE]

Do not output anything else

TAXONOMY DEFINITIONS AND EXAMPLES

1. Entity Reference – multiple entities share the same name.
   e.g., Ambiguous: "Who won the US Open?"
        Disambig: tennis vs golf → Facet: Sport Type → [TYPE]Entity Reference[/TYPE]

2. Part of Entity Reference – one entity has distinct parts/versions.
   e.g., "When did the Olympics happen?" → Women’s 100 m vs Men’s Marathon → [TYPE]Part of Entity Reference[/TYPE]

3. Relationships Between Entities – same entities, different relationships.
   e.g., "Who plays Lara Croft?" → child vs adult version → [TYPE]Relationships Between Entities[/TYPE]

4. Underspecified Common Nouns – vague common noun within a class.
   e.g., "What is Taylor Swift’s new album?" → live vs remix → [TYPE]Underspecified Common Nouns[/TYPE]

5. Degree of an Action – ambiguity in how much/extent an action occurs.
   e.g., "When did the gold standard end?" → partly vs completely → [TYPE]Degree of an Action[/TYPE]

6. Means of an Action – ambiguity in how an action is done.
   e.g., "How was the Garbage Patch found?" → satellite data vs in-person → [TYPE]Means of an Action[/TYPE]

7. Output Type – ambiguity in the expected answer form.
   e.g., "When did the runner-up stop being VP?" → year vs event → [TYPE]Output Type[/TYPE]

8. Temporal Dependency – ambiguity in timeframe.
   e.g., "How long is the term for Texas Governor?" → 1876 vs today → [TYPE]Temporal Dependency[/TYPE]

9. Geographical Dependency – ambiguity by region or location.
   e.g., "When was Tomb Raider released?" → UK vs Berlin → [TYPE]Geographical Dependency[/TYPE]

10. Information Source Dependency – ambiguity depending on data source.
    e.g., "Where does India rank in the economy?" → IMF vs World Bank → [TYPE]Information Source Dependency[/TYPE]


Again, choose ONE taxonomy label from the list above that best describes this ambiguity.

Now classify this case.

Ambiguous Question:
{ambig_q}

Disambiguated Question:
{disambig_q}

Facet:
{facet}

"""


        # ---- Generate model response ----
        inputs = tokenizer(label_prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.0,
                do_sample=False,
            )

        prompt_len = inputs["input_ids"].shape[1]
        new_tokens = outputs[0, prompt_len:]
        text = tokenizer.decode(new_tokens, skip_special_tokens=True)

        print(f"\n--- FACET: {facet} ---")
        print(text)
        print("------------------------\n")

        reasoning = extract_reasoning(text)
        taxonomy_label = extract_label(text)

        row_facet_results.append({
            "facet": facet,
            "taxonomy": taxonomy_label,
            "reasoning": reasoning
        })

    facet_taxonomies_all.append(row_facet_results)


# ================================
# 🔹 Store in DataFrame
# ================================

facet_taxonomies_clean = []
taxonomy_reasoning_clean = []

for lst in facet_taxonomies_all:
    facet_pairs = []
    reasoning_pairs = []
    for d in lst:
        tax = clean_taxonomy_value(d["taxonomy"])
        facet_pairs.append({
            "facet": d["facet"],
            "taxonomy": tax
        })
        reasoning_pairs.append({
            "facet": d["facet"],
            "taxonomy": tax,
            "reasoning": d["reasoning"]
        })
    facet_taxonomies_clean.append(facet_pairs)
    taxonomy_reasoning_clean.append(reasoning_pairs)

df["facet_taxonomies"] = facet_taxonomies_clean          # facet + taxonomy
df["taxonomy_reasoning"] = taxonomy_reasoning_clean      # facet + taxonomy + reasoning
df["taxonomy"] = df["facet_taxonomies"].apply(lambda lst: [d["taxonomy"] for d in lst])


  0%|          | 0/50 [00:00<?, ?it/s]


--- FACET: Type of Show ---
Reasoning: The original question is ambiguous because it could refer to the full series premiere or the first appearance of the characters in a shorter format. The disambiguated version clarifies that it's about the first airing as an animated short on the Tracey Ullman Show.
[TYPE]Part of Entity Reference[/TYPE]
``` Reasoning: The original question is ambiguous because it could refer to the full series premiere or the first appearance of the characters in a shorter format. The disamb
------------------------



  2%|▏         | 1/50 [00:06<05:19,  6.52s/it]


--- FACET: Platform ---
Reasoning: The original question is ambiguous because it doesn't specify the type of platform (e.g., half-hour prime time show, short cartoon, etc.). The disambiguated version clarifies that it's referring to a half-hour prime time show, which changes the context and potentially the date of the first airing.
[TYPE]Output Type[/TYPE]
``` Reasoning: The original question is ambiguous because it doesn't specify the type of platform (e.g., half-hour prime time show
------------------------


--- FACET: State ---
Reasoning: The original question is ambiguous because it does not specify which state's legal age of marriage is being asked about. The disambiguated version clarifies that it refers to the legal age in all but two states, which changes the scope of the answer.
[TYPE]Geographical Dependency[/TYPE]
``` Reasoning: The original question is ambiguous because it does not specify which state's legal age of marriage is being asked about. The disambiguated version 

  6%|▌         | 3/50 [00:13<03:12,  4.09s/it]

...


--- FACET: Season ---
Reasoning: The original question is ambiguous because it does not specify which season of "All in the Family" the character Lionel was played. The disambiguated version clarifies the time period, which is the season of the show.
[TYPE]Temporal Dependency[/TYPE] Reasoning: The original question is ambiguous because it does not specify which season of "All in the Family" the character Lionel was played. The disambiguated version clarifies the time period, which is the season
------------------------


--- FACET: Origin Aspect ---
Reasoning: The original question is ambiguous because it could refer to the origin of the phrase "Just Do It" in terms of its creation or the context in which it was first used. The disambiguated version clarifies that it is asking about the specific setting where the slogan was first used, narrowing down the scope of the query.
[TYPE]Output Type[/TYPE] Reasoning: The original question is ambiguous because it could refer to the origin of t

 98%|█████████▊| 49/50 [02:04<00:03,  3.23s/it]


--- FACET: Inspiration Source ---
Final Taxonomy Label:

[TYPE]Information Source Dependency[/TYPE]
Reasoning: The original question asks about the origin of the saying "Just Do It," which could be interpreted as seeking information from various sources such as advertising history, cultural context, or specific individuals. The disambiguated version narrows down the focus to the inspiration behind the slogan, potentially leading to a more specific answer. However, the core ambiguity remains about the source of the information, which could vary depending on the data
------------------------


--- FACET: Event Type ---
Reasoning: The original question is ambiguous because it could refer to when Brazil organized its first World Cup as the host country or when it bid for hosting rights. The disambiguated version clarifies that the question is about the bidding process rather than the actual hosting.
[TYPE]Event Type[/TYPE]
```
------------------------



100%|██████████| 50/50 [02:09<00:00,  2.60s/it]


--- FACET: Organizer Role ---
Reasoning: The original question is ambiguous because it doesn't specify which organization was responsible for organizing the World Cup. The disambiguated version clarifies that it was FIFA, which is the official organizer of the World Cup.
[TYPE]Organizer Role[/TYPE]
```plaintext
Reasoning: The original question is ambiguous because it doesn't specify which organization was responsible for organizing the World Cup. The disambiguated version clarifies that it was FIFA, which is the official organizer of
------------------------



In [ ]:
df.to_csv('/content/drive/MyDrive/cs546_ambigqa_w_facet.csv', index=False)

In [ ]:
# trim cause it takes so long TT
df = df[:1000]

NEW code with better error handling and prompt

In [ ]:
import re
import pandas as pd
from tqdm import tqdm

# ================================
# 🔹 Valid Taxonomies (for validation)
# ================================
VALID_TAXONOMIES = [
    "Entity Reference",
    "Part of Entity Reference",
    "Relationships Between Entities",
    "Underspecified Common Nouns",
    "Degree of an Action",
    "Means of an Action",
    "Output Type",
    "Temporal Dependency",
    "Geographical Dependency",
    "Information Source Dependency"
]

# ================================
# 🔹 Helper Functions
# ================================

def extract_reasoning(text):
    """
    Extract reasoning section starting with 'Reasoning:' and ending before the [TYPE] tag.
    """
    m = re.search(r"Reasoning\s*:\s*(.*?)(?=\[TYPE|\Z)", text, flags=re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else "N/A"


def extract_label(text):
    """
    Extracts the taxonomy label inside [TYPE]...[/TYPE].
    Handles cases with or without quotes.
    """
    m = re.search(r"\[TYPE\](.*?)\[/TYPE\]", text, flags=re.DOTALL | re.IGNORECASE)
    if not m:
        return "N/A"

    label = m.group(1).strip()
    # Remove quotes if present
    label = label.strip('"').strip("'")
    return label


def clean_taxonomy_value(t):
    """
    Clean up taxonomy strings to remove brackets, quotes, or stray characters.
    """
    if isinstance(t, str):
        return t.replace('[', '').replace(']', '').replace('"', '').replace("'", '').strip()
    return t


# ================================
# 🔹 Taxonomy Classification Loop
# ================================

facet_taxonomies_all = []
invalid_classifications = []  # Track invalid results for review

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Classifying taxonomies"):
    # Skip non-ambiguous questions or those without facets
    if not row.get("is_ambiguous") or not isinstance(row.get("facets"), list) or len(row["facets"]) == 0:
        facet_taxonomies_all.append([])
        continue

    ambig_q = row["question"]
    # Use ALL disambiguated questions (not individual ones)
    disambig_qs = "\n".join([f"{i+1}. {q}" for i, q in enumerate(row.get("disambig_questions", []))])

    row_facet_results = []

    # Process each facet
    for facet in row["facets"]:
        if not facet or facet == 'N/A':
            row_facet_results.append({
                "facet": facet,
                "taxonomy": "N/A",
                "reasoning": "N/A"
            })
            continue

        label_prompt = f"""You are a linguistics expert classifying ambiguity questions and facets according to the given taxonomy definitions.

Your task:
1. Compare the ambiguous question and its disambiguated versions.
2. Identify how the facet explains the differences between them.
3. Choose EXACTLY ONE taxonomy label from the 10 options below.

IMPORTANT RULES:
- You MUST choose from one of the 10 labels listed below
- Do not create new labels
- Do not combine labels
- First provide reasoning, then the label

TAXONOMY DEFINITIONS AND EXAMPLES

1. Entity Reference – multiple entities share the same name.
   Example: Ambiguous: "Who won the US Open?"
            Disambiguated: "Who won the US Open Tennis?" vs "Who won the US Open Golf?"
            Facet: Sport Type → [TYPE]Entity Reference[/TYPE]

2. Part of Entity Reference – one entity has distinct parts/versions.
   Example: "When did the Olympics happen?"
            Disambiguated: "Women's 100m" vs "Men's Marathon"
            Facet: Event Category → [TYPE]Part of Entity Reference[/TYPE]

3. Relationships Between Entities – same entities, different relationships.
   Example: "Who plays Lara Croft in Tomb Raider?"
            Disambiguated: "as a child" vs "as an adult"
            Facet: Age/Role → [TYPE]Relationships Between Entities[/TYPE]

4. Underspecified Common Nouns – vague common noun within a class.
   Example: "What is Taylor Swift's new album?"
            Disambiguated: "live album" vs "remix album"
            Facet: Album Type → [TYPE]Underspecified Common Nouns[/TYPE]

5. Degree of an Action – ambiguity in how much/extent an action occurs.
   Example: "When did the gold standard end in the US?"
            Disambiguated: "began to end" vs "completely ended"
            Facet: Degree → [TYPE]Degree of an Action[/TYPE]

6. Means of an Action – ambiguity in how an action is done.
   Example: "When was the Pacific Garbage Patch found?"
            Disambiguated: "by hypothesis" vs "in person"
            Facet: Method of Discovery → [TYPE]Means of an Action[/TYPE]

7. Output Type – ambiguity in the expected answer form.
   Example: "When did the runner-up stop becoming VP?"
            Disambiguated: "In what year?" vs "What event caused it?"
            Facet: Answer Type → [TYPE]Output Type[/TYPE]

8. Temporal Dependency – ambiguity in timeframe.
   Example: "How long is the term for Texas Governor?"
            Disambiguated: "1876-1972" vs "1972-now"
            Facet: Time Period → [TYPE]Temporal Dependency[/TYPE]

9. Geographical Dependency – ambiguity by region or location.
   Example: "When was Tomb Raider released?"
            Disambiguated: "in UK" vs "in Berlin"
            Facet: Region → [TYPE]Geographical Dependency[/TYPE]

10. Information Source Dependency – ambiguity depending on data source.
    Example: "Where does India rank in the world economy?"
             Disambiguated: "according to IMF" vs "according to World Bank"
             Facet: Source → [TYPE]Information Source Dependency[/TYPE]

REQUIRED OUTPUT FORMAT:
Reasoning: [Your 1-2 sentence explanation here]
[TYPE]Exact Taxonomy Name[/TYPE]

Valid taxonomy names (choose EXACTLY ONE):
- Entity Reference
- Part of Entity Reference
- Relationships Between Entities
- Underspecified Common Nouns
- Degree of an Action
- Means of an Action
- Output Type
- Temporal Dependency
- Geographical Dependency
- Information Source Dependency

Now classify this case.

Ambiguous Question:
{ambig_q}

All Disambiguated Questions:
{disambig_qs}

Facet to classify:
{facet}

Remember: Output reasoning first, then choose EXACTLY ONE label from the 10 options above in the format [TYPE]label[/TYPE].
"""

        # ---- Generate model response ----
        inputs = tokenizer(label_prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.0,  # Deterministic for reproducibility
                do_sample=False,
            )

        prompt_len = inputs["input_ids"].shape[1]
        new_tokens = outputs[0, prompt_len:]
        text = tokenizer.decode(new_tokens, skip_special_tokens=True)

        # Extract reasoning and taxonomy label
        reasoning = extract_reasoning(text)
        taxonomy_label = extract_label(text)

        # Validate taxonomy label
        if taxonomy_label not in VALID_TAXONOMIES:
            print(f"\n⚠️ INVALID TAXONOMY: '{taxonomy_label}'")
            print(f"   Question: {ambig_q}")
            print(f"   Facet: {facet}")
            print(f"   Model output: {text}")
            print("------------------------")

            invalid_classifications.append({
                'row_idx': idx,
                'question': ambig_q,
                'facet': facet,
                'predicted_taxonomy': taxonomy_label,
                'model_output': text
            })

        row_facet_results.append({
            "facet": facet,
            "taxonomy": taxonomy_label,
            "reasoning": reasoning,
            "is_valid": taxonomy_label in VALID_TAXONOMIES
        })

    facet_taxonomies_all.append(row_facet_results)


# ================================
# 🔹 Store in DataFrame
# ================================

facet_taxonomies_clean = []
taxonomy_reasoning_clean = []

for lst in facet_taxonomies_all:
    facet_pairs = []
    reasoning_pairs = []
    for d in lst:
        tax = clean_taxonomy_value(d.get("taxonomy", "N/A"))
        facet_pairs.append({
            "facet": d.get("facet", "N/A"),
            "taxonomy": tax
        })
        reasoning_pairs.append({
            "facet": d.get("facet", "N/A"),
            "taxonomy": tax,
            "reasoning": d.get("reasoning", "N/A"),
            "is_valid": d.get("is_valid", False)
        })
    facet_taxonomies_clean.append(facet_pairs)
    taxonomy_reasoning_clean.append(reasoning_pairs)

df["facet_taxonomies"] = facet_taxonomies_clean          # facet + taxonomy
df["taxonomy_reasoning"] = taxonomy_reasoning_clean      # facet + taxonomy + reasoning
df["taxonomy"] = df["facet_taxonomies"].apply(lambda lst: [d["taxonomy"] for d in lst])

# ================================
# 🔹 Generate Report on Invalid Classifications
# ================================

print(f"\n{'='*60}")
print(f"CLASSIFICATION SUMMARY")
print(f"{'='*60}")
print(f"Total rows processed: {len(df)}")
print(f"Ambiguous questions: {df['is_ambiguous'].sum()}")
print(f"Invalid classifications: {len(invalid_classifications)}")

if invalid_classifications:
    print(f"\n⚠️ WARNING: {len(invalid_classifications)} invalid taxonomy labels detected!")
    print("These need manual review. Saving to 'invalid_taxonomies.csv'...")

    invalid_df = pd.DataFrame(invalid_classifications)
    invalid_df.to_csv('invalid_taxonomies.csv', index=False)
    print("Saved to: invalid_taxonomies.csv")
else:
    print("\n✅ All taxonomy classifications are valid!")

# Calculate validation rate
total_classifications = sum(len(lst) for lst in facet_taxonomies_all)
valid_classifications = sum(d.get("is_valid", False) for lst in facet_taxonomies_all for d in lst)

if total_classifications > 0:
    validation_rate = (valid_classifications / total_classifications) * 100
    print(f"\nValidation Rate: {validation_rate:.2f}%")
    print(f"Valid: {valid_classifications}/{total_classifications}")

Classifying taxonomies: 100%|██████████| 5/5 [00:00<00:00, 2759.77it/s]


CLASSIFICATION SUMMARY
Total rows processed: 5
Ambiguous questions: 4
Invalid classifications: 0

✅ All taxonomy classifications are valid!


In [ ]:
df.to_csv('/content/drive/MyDrive/cs546_dataset_1k.csv', index=False)

old tax code prediction

In [ ]:
pd.set_option('display.max_colwidth', None)
df

,question,type,qaPairs,disambig_questions,is_ambiguous,facets,facet_taxonomies,taxonomy_reasoning,taxonomy
0,When did the simpsons first air on television?,multipleQAs,"[{'question': ['When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?', 'When did the Simpsons first air as a half-hour prime time show?'], 'answer': [['April 19, 1987'], ['December 17, 1989']]}]","[When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?, When did the Simpsons first air as a half-hour prime time show?]",True,"[Type of Show, Platform]","[{'facet': 'Type of Show', 'taxonomy': 'Part of Entity Reference'}, {'facet': 'Platform', 'taxonomy': 'Output Type'}]","[{'facet': 'Type of Show', 'taxonomy': 'Part of Entity Reference', 'reasoning': 'The original question is ambiguous because it could refer to the full series premiere or the first appearance of the characters in a shorter format. The disambiguated version clarifies that it's about the first airing as an animated short on the Tracey Ullman Show.'}, {'facet': 'Platform', 'taxonomy': 'Output Type', 'reasoning': 'The original question is ambiguous because it doesn't specify the type of platform (e.g., half-hour prime time show, short cartoon, etc.). The disambiguated version clarifies that it's referring to a half-hour prime time show, which changes the context and potentially the date of the first airing.'}]","[Part of Entity Reference, Output Type]"
1,Who played george washington in the john adams series?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],[],[],[]
2,What is the legal age of marriage in usa?,multipleQAs,"[{'question': ['What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?', 'What is the legal age of marriage, without parental consent or other authorization, in Nebraska?', 'What is the legal age of marriage, without parental consent or other authorization, in Mississippi?', 'What is youngest legal age of marriage possible in some US states when circumstances permit?'], 'answer': [['18 years of age', '18'], ['19'], ['21'], ['0']]}]","[What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?, What is the legal age of marriage, without parental consent or other authorization, in Nebraska?, What is the legal age of marriage, without parental consent or other authorization, in Mississippi?, What is youngest legal age of marriage possible in some US states when circumstances permit?]",True,"[State, Circumstances for Marriage]","[{'facet': 'State', 'taxonomy': 'Geographical Dependency'}, {'facet': 'Circumstances for Marriage', 'taxonomy': 'Circumstances for an Event'}]","[{'facet': 'State', 'taxonomy': 'Geographical Dependency', 'reasoning': 'The original question is ambiguous because it does not specify which state's legal age of marriage is being asked about. The disambiguated version clarifies that it refers to the legal age in all but two states, which changes the scope of the answer.'}, {'facet': 'Circumstances for Marriage', 'taxonomy': 'Circumstances for an Event', 'reasoning': 'The original question is ambiguous because it does not specify under what circumstances the legal age of marriage applies. The disambiguated version clarifies that it refers to the age without parental consent or other authorization, which changes the specific requirement being asked about.'}]","[Geographical Dependency, Circumstances for an Event]"
3,Who starred in barefoot in the park on broadway?,multipleQAs,"[{'question': ['Who starred in barefoot in the park on broadway as Corie Bratter?', 'Who starred in barefoot in the park on broadway as Victor Velasco?', 'Who starred in barefoot in the park on broadway as Mrs. Banks?', 'Who starred in barefoot in the park on broadway as Paul Bratter?', 'Who starred in barefoot in the park on broadway as Telephone Man?', 'Who starred in barefoot in the park 

new tax code prediction

In [ ]:
len(df)

1000

In [ ]:
pd.set_option('display.max_colwidth', None)
df

,question,type,qaPairs,disambig_questions,is_ambiguous,facets
0,When did the simpsons first air on television?,multipleQAs,"[{'question': ['When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?', 'When did the Simpsons first air as a half-hour prime time show?'], 'answer': [['April 19, 1987'], ['December 17, 1989']]}]","[When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?, When did the Simpsons first air as a half-hour prime time show?]",True,"[Type of Show, Platform]"
1,Who played george washington in the john adams series?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[]
2,What is the legal age of marriage in usa?,multipleQAs,"[{'question': ['What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?', 'What is the legal age of marriage, without parental consent or other authorization, in Nebraska?', 'What is the legal age of marriage, without parental consent or other authorization, in Mississippi?', 'What is youngest legal age of marriage possible in some US states when circumstances permit?'], 'answer': [['18 years of age', '18'], ['19'], ['21'], ['0']]}]","[What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?, What is the legal age of marriage, without parental consent or other authorization, in Nebraska?, What is the legal age of marriage, without parental consent or other authorization, in Mississippi?, What is youngest legal age of marriage possible in some US states when circumstances permit?]",True,"[State, Circumstances for Marriage]"
3,Who starred in barefoot in the park on broadway?,multipleQAs,"[{'question': ['Who starred in barefoot in the park on broadway as Corie Bratter?', 'Who starred in barefoot in the park on broadway as Victor Velasco?', 'Who starred in barefoot in the park on broadway as Mrs. Banks?', 'Who starred in barefoot in the park on broadway as Paul Bratter?', 'Who starred in barefoot in the park on broadway as Telephone Man?', 'Who starred in barefoot in the park on broadway as Delivery Man?'], 'answer': [['Elizabeth Ashley'], ['Kurt Kasznar'], ['Mildred Natwick'], ['Robert Redford'], ['Herbert Edelman'], ['Joseph Keating']]}]","[Who starred in barefoot in the park on broadway as Corie Bratter?, Who starred in barefoot in the park on broadway as Victor Velasco?, Who starred in barefoot in the park on broadway as Mrs. Banks?, Who starred in barefoot in the park on broadway as Paul Bratter?, Who starred in barefoot in the park on broadway as Telephone Man?, Who starred in barefoot in the park on broadway as Delivery Man?]",True,[Character Name]
4,When did the manhattan project began and end?,multipleQAs,"[{'question': ['Based on the initial thoughts of the project, when did the Manhattan project begin and end?', 'Based on when the project was being worked on, when did the Manhattan project begin and end?'], 'answer': [['Began 1939, end 1946'], ['Began 1942, end 1946']]}]","[Based on the initial thoughts of the project, when did the Manhattan project begin and end?, Based on when the project was being worked on, when did the Manhattan project begin and end?]",True,[Time Frame Interpretation]
5,When did the frozen ride open at epcot?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[]
6,Name the landforms that form the boundaries of the peninsular plateau?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[]
7,When was the last time uga won a national championship?,multipleQAs,"[{'question': ['When was the last time UGA won a national football championship?', 'When was the last time UGA won a national gymnastics championship?', 'When was the last time UGA won a national baseball championship?', 'When was the last time UGA won a national golf championship?', 'When was the last time UGA won a national women's swimming and diving championship?', 'When was the last time UGA 

In [ ]:
# import re
# import pandas as pd
# from tqdm import tqdm  # optional progress bar

# # --- Helper functions ---
# def extract_reasoning(text):
#     m = re.search(r"Reasoning\s*:\s*(.*?)(?=\[TYPE|\Z)", text, flags=re.DOTALL | re.IGNORECASE)
#     return m.group(1).strip() if m else "N/A"

# def extract_label(text):
#     # Find the first [TYPE]...[/TYPE] block (case-insensitive)
#     m = re.search(r"\[TYPE\].*?\[/TYPE\]", text, flags=re.DOTALL | re.IGNORECASE)
#     if not m:
#         return "N/A"

#     block = m.group(0)
#     # Extract the inner label contents, tolerant to quotes and case
#     m2 = re.search(
#         r"\[TYPE\]\s*['\"]?([^'\"]+?)['\"]?\s*\[/TYPE\]",
#         block,
#         flags=re.IGNORECASE
#     )
#     if m2:
#         return m2.group(1).strip()
#     # Fallback: strip brackets manually
#     return re.sub(r"\[/?TYPE\]", "", block, flags=re.IGNORECASE).strip()

# # --- Run taxonomy typing per facet ---
# facet_taxonomies_all = []

# for idx, row in tqdm(df.iterrows(), total=len(df)):
#     if not row["is_ambiguous"] or not isinstance(row["facets"], list) or len(row["facets"]) == 0:
#         facet_taxonomies_all.append([])
#         continue

#     ambig_q = row["question"]
#     row_facet_results = []

#     for facet in row["facets"]:
#       # ✅ ENHANCED TAXONOMY PROMPT (faithful to AmbigChat Table 1)
#       label_prompt = f"""
# You are a linguistics expert predicting ambigious taxonomy labels for ambiguous questions and facets according to the taxonomy definitions.
# Facet Definition:
# A facet is the specific dimension or aspect along which the meaning of a question can vary — such as time, location, source, or entity type. It describes what changes between different possible interpretations of the question.


# Your task:
# 1. Look at the ambiguous question and the facet
# 2. Choose ONE taxonomy label from the list below that best matches
# 3. First, explain briefly (1–2 sentences) why. Start your explanation with 'Reasoning:'.
# 4. Then output the final label exactly once in this format:
# [TYPE]["<Taxonomy Name>"][/TYPE]
# 5. Do not output anything else after the final taxonomy label.

# Taxonomy Options and Examples:
# Question Nouns
# 1. Entity Reference – Multiple entities, such as people, places, and events, can share the same name.
# Example: Facet “Sport Type” → the name “US Open” could mean tennis or golf.
# Final Answer: [TYPE]["Entity Reference"][/TYPE]

# 2. Part of Entity Reference – One entity can have different parts or variations.
# Example: Facet “Event Category” → “Paris Olympics” could mean Women’s 100 m or Men’s Marathon.
# Final Answer: [TYPE]["Part of Entity Reference"][/TYPE]

# 3. Relationships Between Entities – The same set of entities can have different relationships with each other.
# Example: Facet “Character Age Version” → “Who plays Lara Croft” could mean the child or adult version.
# Final Answer: [TYPE]["Relationships Between Entities"][/TYPE]

# 4. Underspecified Common Nouns – Common nouns define a class of objects and can be underspecified.
# Example: Facet “Album Type” → “new album” could mean live or remix.
# Final Answer: [TYPE]["Underspecified Common Nouns"][/TYPE]

# Question Verbs
# 5. Degree of an Action – ambiguity in how much or to what extent an action occurs.
# Example: Facet “Extent of End” → “When did the gold standard end” could mean partly or completely.
# Final Answer: [TYPE]["Degree of an Action"][/TYPE]

# 6. Means of an Action – ambiguity in how an action is performed.
# Example: Facet “Method of Discovery” → “When was the Garbage Patch found” could mean found by data vs in person.
# Final Answer: [TYPE]["Means of an Action"][/TYPE]

# Adverbs
# 7. Output Type – ambiguity in what kind of answer the question expects.
# Example: Facet “Answer Form” → “When did the runner-up stop being VP” could expect a year or an event.
# Final Answer: [TYPE]["Output Type"][/TYPE]

# Question Scope
# 8. Temporal Dependency – ambiguity in the timeframe in which the question situates.
# Example: Facet “Time Frame” → “How long is the term for Texas Governor” could mean in 1876 vs today.
# Final Answer: [TYPE]["Temporal Dependency"][/TYPE]

# 9. Geographical Dependency – ambiguity in the Gographical location or region in which the question situates.
# Example: Facet “Location of Release” → “When was Tomb Raider released” could mean Berlin vs UK.
# Final Answer: [TYPE]["Geographical Dependency"][/TYPE]

# 10. Information Source Dependency – ambiguity depending on which information source is referenced
# Example: Facet “Information Source” → “Where does India rank in the economy” could mean IMF vs World Bank.
# Final Answer: [TYPE]["Information Source Dependency"][/TYPE]

# Now classify for this :

# Ambiguous Question: {ambig_q}
# Facet: {facet}

# """
#       inputs = tokenizer(label_prompt, return_tensors="pt").to(device)
#       with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=100,
#                 temperature=0.0,
#                 do_sample=False,
#             )
#       prompt_len = inputs["input_ids"].shape[1]
#       new_tokens = outputs[0, prompt_len:]
#       text = tokenizer.decode(new_tokens, skip_special_tokens=True)

#       # 👀 PRINT RAW MODEL OUTPUT HERE
#       print(f"\n--- FACET: {facet} ---")
#       print(text)
#       print("------------------------\n")

#       reasoning = extract_reasoning(text)
#       taxonomy_label = extract_label(text)

#       row_facet_results.append({
#             "facet": facet,
#             "taxonomy": taxonomy_label,
#             "reasoning": reasoning
#         })
#     facet_taxonomies_all.append(row_facet_results)

# # --- Store in dataframe ---
# df = df.copy()
# def clean_taxonomy_value(t):
#     if isinstance(t, str):
#         return t.replace('[', '').replace(']', '').replace('"', '').replace("'", '').strip()
#     return t

# facet_taxonomies_clean = []
# taxonomy_reasoning_clean = []

# for lst in facet_taxonomies_all:
#     facet_pairs = []
#     reasoning_pairs = []
#     for d in lst:
#         tax = clean_taxonomy_value(d["taxonomy"])
#         facet_pairs.append({
#             "facet": d["facet"],
#             "taxonomy": tax
#         })
#         reasoning_pairs.append({
#             "facet": d["facet"],
#             "taxonomy": tax,
#             "reasoning": d["reasoning"]
#         })
#     facet_taxonomies_clean.append(facet_pairs)
#     taxonomy_reasoning_clean.append(reasoning_pairs)

# df["facet_taxonomies"] = facet_taxonomies_clean          # facet + taxonomy
# df["taxonomy_reasoning"] = taxonomy_reasoning_clean      # facet + taxonomy + reasoning
# df["taxonomy"] = df["facet_taxonomies"].apply(lambda lst: [d["taxonomy"] for d in lst])
# # df["reasoning"] = df["facet_taxonomies"].apply(
# #     lambda lst: "; ".join([f'Facet: {d["facet"]} → {d["taxonomy"]}' for d in lst])
# # )

In [ ]:
# pd.set_option('display.max_colwidth', None)
# df

CONTRASTIVE PAIRS GENERATION

In [ ]:
import pandas as pd
df=pd.read_csv('dataset_1k.csv')
df=df[:5]
df.head()

,question,type,qaPairs,disambig_questions,is_ambiguous,facets,facet_taxonomies,taxonomy_reasoning,taxonomy
0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,['When did the Simpsons first air on televisio...,True,"['Type of Show', 'Platform']","[{'facet': 'Type of Show', 'taxonomy': 'Part o...","[{'facet': 'Type of Show', 'taxonomy': 'Part o...","['Part of Entity Reference', 'Part of Entity R..."
1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],[],[],[]
2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,"['What is the legal age of marriage, without p...",True,"['State', 'Circumstances for Marriage']","[{'facet': 'State', 'taxonomy': 'Geographical ...","[{'facet': 'State', 'taxonomy': 'Geographical ...","['Geographical Dependency', 'Geographical Depe..."
3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,['Who starred in barefoot in the park on broad...,True,['Character Name'],"[{'facet': 'Character Name', 'taxonomy': 'Rela...","[{'facet': 'Character Name', 'taxonomy': 'Rela...",['Relationships Between Entities']
4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,['Based on the initial thoughts of the project...,True,['Time Frame Interpretation'],"[{'facet': 'Time Frame Interpretation', 'taxon...","[{'facet': 'Time Frame Interpretation', 'taxon...",['Temporal Dependency']


In [ ]:
# ==============================================================
# 🔹 IMPORTS & SETUP
# ==============================================================
import re
import ast
import gc
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---- Device Setup ----
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


# ==============================================================
# 🔹 Helper: Extract only [OUTPUT] content
# ==============================================================
def extract_output(text):
    """
    Extracts the model-generated clarifying question from between [OUTPUT] tags.
    """
    m = re.search(r"\[OUTPUT\](.*?)\[/OUTPUT\]", text, flags=re.DOTALL)
    if m:
        return m.group(1).strip()
    else:
        # fallback — return last few lines if no tags
        lines = text.strip().splitlines()
        return lines[-1].strip() if lines else text.strip()

# ==============================================================
# 🔹 Build Positive Clarifying Prompt
# ==============================================================
def build_positive_prompt(question, facets, taxonomies):
    facets_txt = ", ".join(facets) if facets else "N/A"
    tax_txt = ", ".join(taxonomies) if taxonomies else "N/A"

    return f"""
You are a helpful assistant that clarifies ambiguous questions.

Your task: Ask a **single-turn clarifying question** that helps the user
specify their intent and resolve all the ambiguity.

---

**Ambiguous Question:**
{question}

**Facets (dimensions of ambiguity):**
{facets_txt}

**Taxonomy Types (nature of ambiguity):**
{tax_txt}

---

Guidelines:
1. Ask one polite, natural question that helps clarify **all listed facets**.
2. Use the taxonomy types to understand what kind of clarification to ask.
3. Do **not** answer the question.
4. Keep your response short (1 sentence max).
5. Output the clarifying question only between [OUTPUT] and [/OUTPUT].

---

### Example 1
Ambiguous: "When did The Simpsons first air?"
Facets: Type of Show, Broadcast Region
Taxonomy: Part of Entity Reference, Geographical Dependency
Output: [OUTPUT]Could you clarify which type of Simpsons show you’re referring to, and in which region it aired?[/OUTPUT]

### Example 2
Ambiguous: "Who won the US Open?"
Facets: Sport Type, Year
Taxonomy: Entity Reference, Temporal Dependency
Output: [OUTPUT]Do you mean the US Open in tennis or golf, and which year are you asking about?[/OUTPUT]

---

Now write your clarifying question.
Remember to wrap it exactly as [OUTPUT] ... [/OUTPUT].
"""

# ==============================================================
# 🔹 LLM Call Utility
# ==============================================================
def generate_text(prompt, temperature=0.5, max_new_tokens=100, debug=False):
    """
    Generates only the model's clarifying question, not the prompt.
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=3000).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=True
        )

    # Decode only the newly generated part, not the input prompt
    gen_tokens = outputs[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(gen_tokens, skip_special_tokens=True)

    if debug:
        print("\n=== RAW MODEL OUTPUT ===\n", text, "\n=========================\n")

    return extract_output(text)

# ==============================================================
# 🔹 MAIN LOOP — GENERATE POSITIVE RESPONSES ONLY
# ==============================================================
positive_responses = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating positive clarifying questions"):
    if not row.get("is_ambiguous", False):
        positive_responses.append("N/A")
        continue

    question = row["question"]

    # --- Parse facet_taxonomies safely ---
    raw_ft = row.get("facet_taxonomies", [])
    if isinstance(raw_ft, str):
        try:
            raw_ft = ast.literal_eval(raw_ft)
        except Exception:
            raw_ft = []

    facets = [ft.get("facet", "") for ft in raw_ft if isinstance(ft, dict)]
    taxonomies = [ft.get("taxonomy", "") for ft in raw_ft if isinstance(ft, dict)]

    try:
        pos_prompt = build_positive_prompt(question, facets, taxonomies)
        pos_out = generate_text(pos_prompt, temperature=0.6, debug=True)
        positive_responses.append(pos_out)
    except torch.cuda.OutOfMemoryError:
        print(f"\n⚠️ OOM at row {idx}, switching to CPU temporarily...")
        model.to("cpu")
        pos_out = generate_text(pos_prompt, temperature=0.6, debug=False)
        positive_responses.append(pos_out)
        model.to(device)
    finally:
        torch.cuda.empty_cache()
        gc.collect()

    # --- Print summary per question ---
    print(f"\n🟩 Row {idx}")
    print(f"Q: {question}")
    print(f"Facets: {facets}")
    print(f"Taxonomies: {taxonomies}")
    print(f"✅ Clarifying Question: {pos_out}")
    print("------------------------------------------------------\n")

# ==============================================================
# 🔹 SAVE OUTPUT
# ==============================================================
df["positive_response"] = positive_responses
df.to_csv("positive_clarifying_responses.csv", index=False)
print("\n✅ Saved results to positive_clarifying_responses.csv")


Using device: cuda


Generating positive clarifying questions:   0%|          | 0/5 [00:00<?, ?it/s]


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the original TV series or a movie version, and on which network or platform was it first broadcast?[/OUTPUT] 



Generating positive clarifying questions:  20%|██        | 1/5 [00:22<01:28, 22.18s/it]


🟩 Row 0
Q: When did the simpsons first air on television?
Facets: ['Type of Show', 'Platform']
Taxonomies: ['Part of Entity Reference', 'Part of Entity Reference']
✅ Clarifying Question: Are you asking about the original TV series or a movie version, and on which network or platform was it first broadcast?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the legal age of marriage across different states in the USA, or specifically under certain circumstances such as parental consent?[/OUTPUT] 



...


🟩 Row 2
Q: What is the legal age of marriage in usa?
Facets: ['State', 'Circumstances for Marriage']
Taxonomies: ['Geographical Dependency', 'Geographical Dependency']
✅ Clarifying Question: Are you asking about the legal age of marriage across different states in the USA, or specifically under certain circumstances such as parental consent?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]Did you mean the character names or specific actors who performed the roles in Barefoot in the Park on Broadway?[/OUTPUT] 



Generating positive clarifying questions:  80%|████████  | 4/5 [01:05<00:16, 16.29s/it]


🟩 Row 3
Q: Who starred in barefoot in the park on broadway?
Facets: ['Character Name']
Taxonomies: ['Relationships Between Entities']
✅ Clarifying Question: Did you mean the character names or specific actors who performed the roles in Barefoot in the Park on Broadway?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]In which specific context do you refer to the Manhattan Project, and over what time frame are you interested?[/OUTPUT] 



Generating positive clarifying questions: 100%|██████████| 5/5 [01:24<00:00, 16.99s/it]


🟩 Row 4
Q: When did the manhattan project began and end?
Facets: ['Time Frame Interpretation']
Taxonomies: ['Temporal Dependency']
✅ Clarifying Question: In which specific context do you refer to the Manhattan Project, and over what time frame are you interested?
------------------------------------------------------


✅ Saved results to positive_clarifying_responses.csv


In [ ]:
df

,question,type,qaPairs,disambig_questions,is_ambiguous,facets,facet_taxonomies,taxonomy_reasoning,taxonomy,positive_response
0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,['When did the Simpsons first air on televisio...,True,"['Type of Show', 'Platform']","[{'facet': 'Type of Show', 'taxonomy': 'Part o...","[{'facet': 'Type of Show', 'taxonomy': 'Part o...","['Part of Entity Reference', 'Part of Entity R...",Are you asking about the original TV series or...
1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],[],[],[],N/A
2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,"['What is the legal age of marriage, without p...",True,"['State', 'Circumstances for Marriage']","[{'facet': 'State', 'taxonomy': 'Geographical ...","[{'facet': 'State', 'taxonomy': 'Geographical ...","['Geographical Dependency', 'Geographical Depe...",Are you asking about the legal age of marriage...
3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,['Who starred in barefoot in the park on broad...,True,['Character Name'],"[{'facet': 'Character Name', 'taxonomy': 'Rela...","[{'facet': 'Character Name', 'taxonomy': 'Rela...",['Relationships Between Entities'],Did you mean the character names or specific a...
4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,['Based on the initial thoughts of the project...,True,['Time Frame Interpretation'],"[{'facet': 'Time Frame Interpretation', 'taxon...","[{'facet': 'Time Frame Interpretation', 'taxon...",['Temporal Dependency'],In which specific context do you refer to the ...
